# Data Science - Laboratorio 9

## Laboratorio 9. Visualizaciones interactivas y dashboards

**Integrantes**
- Diego Alberto Leiva
- José Pablo Orellana

#### Librerias

In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

#### Dashboard

In [2]:
# Cargar los archivos CSV
file_path_precios = 'data/precios.csv'
df_combustibles = pd.read_csv(file_path_precios)

file_path_consumos = 'data/consumos.csv'
df_consumos = pd.read_csv(file_path_consumos)

# Convertir las fechas a formato datetime
df_combustibles['Fecha'] = pd.to_datetime(df_combustibles['Fecha'], format='%b-%y')
df_consumos['Fecha'] = pd.to_datetime(df_consumos['Fecha'], format='%b-%y')
df_consumos['Año'] = df_consumos['Fecha'].dt.year

# Crear la aplicación Dash
app = dash.Dash(__name__)

# Función para crear la gráfica de barras
def create_figure(selected_year):
    # Filtrar por el año seleccionado
    df_filtered = df_combustibles[df_combustibles['Fecha'].dt.year == selected_year]
    
    # Transformar los datos para la gráfica
    df_melted = df_filtered.melt(
        id_vars=['Fecha'], 
        value_vars=['Gasolina Superior', 'Gasolina Regular', 'Diesel'],
        var_name='Tipo Combustible', 
        value_name='Precios'
    )
    
    # Ordenar los meses
    df_melted['Mes'] = df_melted['Fecha'].dt.strftime('%b')
    meses_espanol = {
        'Jan': 'Ene', 'Feb': 'Feb', 'Mar': 'Mar', 'Apr': 'Abr', 'May': 'May', 'Jun': 'Jun',
        'Jul': 'Jul', 'Aug': 'Ago', 'Sep': 'Sep', 'Oct': 'Oct', 'Nov': 'Nov', 'Dec': 'Dic'
    }
    df_melted['Mes'] = df_melted['Mes'].map(meses_espanol)
    df_melted['Mes'] = pd.Categorical(df_melted['Mes'], categories=[
        'Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 
        'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'], ordered=True)

    # Crear la gráfica de barras
    fig = px.bar(
        df_melted,
        x='Mes',
        y='Precios',
        color='Tipo Combustible',
        facet_col='Tipo Combustible',
        facet_col_spacing=0.05,
        category_orders={"Tipo Combustible": ["Gasolina Regular", "Gasolina Superior", "Diesel"]},  
        labels={ 'Mes': 'Meses', 'Precios': 'Precios (GTQ/Galón)', 'Tipo Combustible': 'Tipo de Combustible' }
    )

    # Modificar colores 
    color_discrete_map = { 'Gasolina Superior': '#158C59', 'Gasolina Regular': '#C22B30', 'Diesel': '#A2A2A2' }
    border_color_map = { 'Gasolina Superior': '#0F623E', 'Gasolina Regular': '#972125', 'Diesel': '#808080' }
    
    # Aplicar el color interno y contorno a las barras
    fig.for_each_trace(
        lambda trace: trace.update(marker_color=color_discrete_map[trace.name], marker_line_color=border_color_map[trace.name], marker_line_width=3, opacity=0.9)
    )

    # Ajustes adicionales de estilo
    fig.update_layout(
        plot_bgcolor='#FAF3E0',
        paper_bgcolor='#FAF3E0',
        title={ 'text': f"", 'y': 0.9, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top' },
        font=dict(color='#006CAF'),
        showlegend=False
    )

    # Cambiar los títulos de las facetas
    fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Tipo de Combustible=", "")))
    fig.for_each_annotation(lambda a: a.update(text=a.text + { 'Gasolina Regular': " (Rojo)", 'Gasolina Superior': " (Verde)", 'Diesel': " (Negro)" }[a.text]))

    return fig

# Función para calcular el promedio de cada tipo de combustible
def calcular_promedio(selected_year):
    df_filtered = df_combustibles[df_combustibles['Fecha'].dt.year == selected_year]
    promedio_superior = df_filtered['Gasolina Superior'].mean()
    promedio_regular = df_filtered['Gasolina Regular'].mean()
    promedio_diesel = df_filtered['Diesel'].mean()
    return round(promedio_superior, 2), round(promedio_regular, 2), round(promedio_diesel, 2)

# Función para obtener datos anuales para la gráfica circular
def get_yearly_data(year):
    year_data = df_consumos[df_consumos['Año'] == year]
    total_regular = year_data['Gasolina regular'].sum()
    total_superior = year_data['Gasolina superior'].sum()
    total_diesel = year_data['Diesel'].sum()
    return total_regular, total_superior, total_diesel

# Layout de la aplicación
app.layout = html.Div([
    # Parte superior: gráficas de barras y cuadros de precios
    html.Div([
        html.Div([
            html.H2("Tendencia de Precios de Combustible por Año", style={'textAlign': 'center', 'fontSize': '28px'}),
            dcc.Dropdown(
                id='year-dropdown-precios',
                options=[{'label': str(year), 'value': year} for year in sorted(df_combustibles['Fecha'].dt.year.unique())],
                value=df_combustibles['Fecha'].dt.year.min(),
                style={'width': '200px', 'margin': 'auto', 'color': '#006CAF'}
            ),
            dcc.Graph(id='price-graph')
        ], style={'width': '65%', 'display': 'inline-block', 'verticalAlign': 'top'}),

        html.Div([
            html.H2("Promedio de Precios de Combustible por Año", style={'textAlign': 'center', 'fontSize': '28px', 'marginTop': '25px'}),
            html.Div(id='cuadros-promedio-precios', style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center', 'gap': '20px'})
        ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top'})
    ], style={'display': 'flex', 'alignItems': 'flex-start', 'padding': '20px'}),

    # Parte inferior: gráfica circular y cuadros de barriles
    html.Div([
        html.Div([
            html.H2('Consumo de Combustibles por Año', style={'textAlign': 'center', 'color': '#006CAF', 'marginTop': '50px'}),  
            dcc.Dropdown(
                id='year-dropdown-consumos',
                options=[{'label': str(year), 'value': year} for year in df_consumos['Año'].unique()],
                value=df_consumos['Año'].max(),  
                style={'width': '50%', 'margin': '20px auto'}  
            ),
            dcc.Graph(id='pie-chart')
        ], style={'width': '65%', 'display': 'inline-block', 'verticalAlign': 'top'}),

        html.Div(id='cuadros-consumos', style={
            'width': '30%', 'display': 'flex', 'flexDirection': 'column', 'justifyContent': 'center', 'alignItems': 'center', 'gap': '20px', 'verticalAlign': 'top'})  
    ], style={'display': 'flex', 'alignItems': 'center', 'padding': '20px', 'marginTop': '50px'})  
], style={'backgroundColor': '#FAF3E0', 'padding': '10px', 'color': '#006CAF'})

# Callback para actualizar los cuadros y la gráfica de barras (precios)
@app.callback(
    [Output('cuadros-promedio-precios', 'children'),
     Output('price-graph', 'figure')],
    [Input('year-dropdown-precios', 'value')]
)
def actualizar_dashboard_precios(selected_year):
    promedio_superior, promedio_regular, promedio_diesel = calcular_promedio(selected_year)
    
    cuadros_precios = [
        html.Div([
            html.Img(src='/assets/img/rojo.jpg', style={'width': '60px', 'height': '60px', 'marginRight': '20px'}),
            html.Span(f"GTQ {promedio_regular}", style={'fontSize': '28px', 'fontWeight': 'bold', 'color': '#C22B30'}),
        ], style={
            'border': '2px solid #972125', 'padding':
                        '10px', 'borderRadius': '5px', 
            'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 
            'width': '350px', 'height': '100px'}),

        html.Div([
            html.Img(src='/assets/img/verde.jpg', style={'width': '60px', 'height': '60px', 'marginRight': '20px'}),  
            html.Span(f"GTQ {promedio_superior}", style={'fontSize': '28px', 'fontWeight': 'bold', 'color': '#158C59'}),
        ], style={
            'border': '2px solid #0F623E', 'padding': '10px', 'borderRadius': '5px', 
            'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 
            'width': '350px', 'height': '100px'}),

        html.Div([
            html.Img(src='/assets/img/negro.jpg', style={'width': '60px', 'height': '60px', 'marginRight': '20px'}),
            html.Span(f"GTQ {promedio_diesel}", style={'fontSize': '28px', 'fontWeight': 'bold', 'color': '#A2A2A2'}),
        ], style={
            'border': '2px solid #808080', 'padding': '10px', 'borderRadius': '5px', 
            'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 
            'width': '350px', 'height': '100px'})
    ]
    
    # Crear la figura con la gráfica de barras
    figura = create_figure(selected_year)

    return cuadros_precios, figura

# Callback para actualizar la gráfica circular y los cuadros de barriles (consumos)
@app.callback(
    [Output('pie-chart', 'figure'),
     Output('cuadros-consumos', 'children')],
    [Input('year-dropdown-consumos', 'value')]
)
def update_chart_consumos(year):
    # Obtener datos del año seleccionado
    total_regular, total_superior, total_diesel = get_yearly_data(year)

    # Convertir los valores a millones de barriles
    total_regular_mill = total_regular / 1e6
    total_superior_mill = total_superior / 1e6
    total_diesel_mill = total_diesel / 1e6

    # Datos para la gráfica circular
    labels = ['Gasolina regular', 'Gasolina superior', 'Diesel']
    values = [total_regular, total_superior, total_diesel]
    colors = ['#2B9666', '#C73F41', '#ABAAA8']

    # Crear la gráfica circular
    fig = go.Figure(data=[go.Pie(labels=labels, values=values, marker=dict(colors=colors), hole=0.3)])
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0), paper_bgcolor='#FAF3E0')

    # Crear los cuadros con el formato proporcionado
    cuadros_consumos = [
        html.Div([
            html.Img(src='/assets/img/rojo.jpg', style={'width': '60px', 'height': '60px', 'marginRight': '20px'}),
            html.Span(f"{total_regular_mill:.2f} Millones de Barriles", style={'fontSize': '24px', 'fontWeight': 'bold', 'color': '#C22B30'}),
        ], style={
            'border': '2px solid #972125', 'padding': '10px', 'borderRadius': '5px',
            'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center',
            'width': '350px', 'height': '100px', 'marginBottom': '20px', 'marginLeft': '20px'}),  

        html.Div([
            html.Img(src='/assets/img/verde.jpg', style={'width': '60px', 'height': '60px', 'marginRight': '20px'}),
            html.Span(f"{total_superior_mill:.2f} Millones de Barriles", style={'fontSize': '24px', 'fontWeight': 'bold', 'color': '#158C59'}),
        ], style={
            'border': '2px solid #0F623E', 'padding': '10px', 'borderRadius': '5px',
            'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center',
            'width': '350px', 'height': '100px', 'marginBottom': '20px', 'marginLeft': '20px'}),  

        html.Div([
            html.Img(src='/assets/img/negro.jpg', style={'width': '60px', 'height': '60px', 'marginRight': '20px'}),
            html.Span(f"{total_diesel_mill:.2f} Millones de Barriles", style={'fontSize': '24px', 'fontWeight': 'bold', 'color': '#A2A2A2'}),
        ], style={
            'border': '2px solid #808080', 'padding': '10px', 'borderRadius': '5px',
            'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center',
            'width': '350px', 'height': '100px', 'marginBottom': '20px', 'marginLeft': '20px'})  
    ]

    return fig, cuadros_consumos

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)



In [12]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd

# Cargar los archivos CSV
consumos = pd.read_csv('data/predictions/consumos_preds.csv', sep=';')
importaciones = pd.read_csv('data/predictions/importaciones_preds.csv', sep=';')
precios = pd.read_csv('data/predictions/precios_preds.csv', sep=';')

# Convertir las fechas a formato datetime
for df in [consumos, importaciones, precios]:
    df['Fecha'] = pd.to_datetime(df['Fecha'])

# Crear la aplicación Dash
app = dash.Dash(__name__)

# Layout de la aplicación
app.layout = html.Div([
    # Cambiar el título con el color azul
    html.H1('Predicciones de Combustibles', style={'color': '#006CAF', 'textAlign': 'center'}),
    
    # Menú desplegable para seleccionar el modelo
    dcc.Dropdown(
        id='model-dropdown',
        options=[
            {'label': 'Modelo Consumos', 'value': 'consumos'},
            {'label': 'Modelo Importaciones', 'value': 'importaciones'},
            {'label': 'Modelo Precios', 'value': 'precios'}
        ],
        value='consumos',
        style={'width': '45%', 'margin': 'auto', 'height': '40px'}  # Ajustamos el ancho y la altura
    ),
    
    # Menú desplegable para seleccionar el tipo de combustible
    dcc.Dropdown(
        id='fuel-dropdown',
        options=[
            {'label': 'Gasolina Regular', 'value': 'Gasolina Regular'},
            {'label': 'Gasolina Superior', 'value': 'Gasolina Superior'},
            {'label': 'Diesel', 'value': 'Diesel'}
        ],
        value='Gasolina Regular',
        style={'width': '45%', 'margin': 'auto', 'marginTop': '10px', 'height': '40px'}  # Ajustamos el ancho y la altura
    ),
    
    # Gráfica
    dcc.Graph(id='predictions-graph')
], style={'backgroundColor': '#FAF3E0', 'padding': '20px'})  # Fondo de toda la página y ajuste de padding

# Callback para actualizar la gráfica en función del modelo y el tipo de combustible seleccionados
@app.callback(
    Output('predictions-graph', 'figure'),
    [Input('model-dropdown', 'value'),
     Input('fuel-dropdown', 'value')]
)
def update_graph(selected_model, selected_fuel):
    # Seleccionar el dataframe basado en el modelo seleccionado
    if selected_model == 'consumos':
        df = consumos
    elif selected_model == 'importaciones':
        df = importaciones
    else:
        df = precios
    
    # Mapear columnas basadas en el combustible seleccionado
    if selected_fuel == 'Gasolina Regular':
        real_column = 'Gasolina Regular'
        train_column = 'Predicciones Entreno Gasolina Regular'
        test_column = 'Predicciones Prueba Gasolina Regular'
    elif selected_fuel == 'Gasolina Superior':
        real_column = 'Gasolina Superior'
        train_column = 'Predicciones Entreno Gasolina Superior'
        test_column = 'Predicciones Prueba Gasolina Superior'
    else:
        real_column = 'Diesel'
        train_column = 'Predicciones Entreno Diesel'
        test_column = 'Predicciones Prueba Diesel'
    
    # Crear la figura
    fig = go.Figure()

    # Añadir la serie real
    fig.add_trace(go.Scatter(x=df['Fecha'], y=df[real_column],
                             mode='lines', name='Serie Real',
                             line=dict(color='blue', width=2)))
    
    # Añadir las predicciones de entrenamiento
    fig.add_trace(go.Scatter(x=df['Fecha'], y=df[train_column],
                             mode='lines', name='Predicciones Entreno',
                             line=dict(color='red', width=2, dash='dash')))
    
    # Añadir las predicciones de prueba
    fig.add_trace(go.Scatter(x=df['Fecha'], y=df[test_column],
                             mode='lines', name='Predicciones Prueba',
                             line=dict(color='green', width=2, dash='dash')))
    
    # Ajustes adicionales de la gráfica
    fig.update_layout(title='Predicciones del modelo',
                      xaxis_title='Fecha',
                      yaxis_title='Valores',
                      plot_bgcolor='#FAF3E0',
                      paper_bgcolor='#FAF3E0',
                      font=dict(color='#006CAF'))
    
    return fig

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)
